# CE49X: Introduction to Computational Thinking and Data Science for Civil Engineers
## Week 1 — Example: Wind Turbine Power Curve Simulation

**Instructor:** Dr. Eyuphan Koc  
**Department of Civil Engineering, Bogazici University**  
**Semester:** Spring 2026

---

## Why This Example?

Wind energy is one of the fastest-growing areas in civil and environmental engineering. Before a wind farm is built, engineers need to predict how much power a turbine will generate at different wind speeds. This relationship is called the **power curve**.

In this notebook, we will build a wind turbine power curve simulation **from scratch** using only the Python fundamentals from this week's lecture:

- Variables, arithmetic, and basic types
- Functions with default parameters
- Control flow (`if`/`elif`/`else`)
- Lists, loops, and list comprehensions
- String formatting for output

> **Example: Civil Engineering**  
> Power curve analysis is essential for wind farm site selection, energy yield estimation, and financial feasibility studies — all core tasks for civil engineers working in renewable energy.

---
## 1. The Physics of Wind Power

The power available in the wind passing through a rotor of area $A$ at speed $v$ is:

$$P_{\text{available}} = \frac{1}{2} \, \rho \, A \, v^3$$

where:
- $\rho$ = air density (typically 1.225 kg/m³ at sea level)
- $A$ = rotor swept area = $\pi R^2$
- $v$ = wind speed (m/s)

A turbine can only capture a fraction of this, characterized by the **power coefficient** $C_p$:

$$P_{\text{turbine}} = \frac{1}{2} \, \rho \, A \, C_p \, v^3$$

> **Definition: Betz Limit**  
> The theoretical maximum $C_p$ is $16/27 \approx 0.593$. No turbine can extract more than ~59.3% of the wind's kinetic energy. Modern turbines achieve $C_p \approx 0.35$–$0.45$.

### Operating Regions of a Wind Turbine

A real turbine doesn't simply follow the cubic law at all speeds. There are **four distinct regions**:

| Region | Wind Speed | Power Output | Reason |
|--------|-----------|--------------|--------|
| I | $v < v_{\text{cut-in}}$ | 0 | Not enough wind to overcome friction |
| II | $v_{\text{cut-in}} \le v \le v_{\text{rated}}$ | Increases with $v^3$ | Normal power production |
| III | $v_{\text{rated}} < v \le v_{\text{cut-out}}$ | $P_{\text{rated}}$ (constant) | Blades pitch to limit power |
| IV | $v > v_{\text{cut-out}}$ | 0 | Turbine shuts down for safety |

> **Key Insight: Why Cut-Out?**  
> At very high wind speeds (typically > 25 m/s), structural loads on the blades and tower become dangerously large. The turbine brakes and feathers its blades to prevent damage.

---
## 2. Defining Turbine Parameters

Let's model a typical modern onshore wind turbine — similar to the Vestas V90-2.0 MW.

In [ ]:
import math

# --- Turbine specifications ---
rotor_diameter = 90        # meters
hub_height = 80            # meters
rated_power = 2000         # kW (2 MW)
cut_in_speed = 4.0         # m/s
rated_speed = 13.0         # m/s
cut_out_speed = 25.0       # m/s

# --- Environmental parameters ---
air_density = 1.225        # kg/m³ (sea-level standard)
power_coefficient = 0.40   # Cp (realistic modern turbine)

# --- Derived quantities ---
rotor_radius = rotor_diameter / 2
swept_area = math.pi * rotor_radius ** 2  # m²

print(f"Rotor diameter:  {rotor_diameter} m")
print(f"Swept area:      {swept_area:,.1f} m²")
print(f"Rated power:     {rated_power} kW ({rated_power/1000} MW)")
print(f"Operating range: {cut_in_speed}–{cut_out_speed} m/s")
print(f"Power coeff Cp:  {power_coefficient}")

---
## 3. Building the Power Curve Function

We now translate the four operating regions into a Python function using `if`/`elif`/`else`.

In [ ]:
def wind_power(v, rho=1.225, A=swept_area, Cp=power_coefficient,
               v_ci=cut_in_speed, v_r=rated_speed, v_co=cut_out_speed,
               P_rated=rated_power):
    """Calculate turbine power output at wind speed v.

    Args:
        v:       Wind speed (m/s)
        rho:     Air density (kg/m³)
        A:       Swept area (m²)
        Cp:      Power coefficient
        v_ci:    Cut-in speed (m/s)
        v_r:     Rated speed (m/s)
        v_co:    Cut-out speed (m/s)
        P_rated: Rated power (kW)

    Returns:
        Power output in kW
    """
    if v < v_ci or v > v_co:
        # Region I or IV — no power
        return 0.0
    elif v <= v_r:
        # Region II — cubic power law
        power = 0.5 * rho * A * Cp * v ** 3 / 1000  # W -> kW
        return min(power, P_rated)  # safety cap
    else:
        # Region III — rated power (blade pitch control)
        return P_rated

In [ ]:
# Quick test at a few speeds
test_speeds = [0, 3, 4, 8, 13, 15, 25, 30]

for v in test_speeds:
    p = wind_power(v)
    print(f"v = {v:5.1f} m/s  →  P = {p:8.1f} kW")

---
## 4. Generating the Full Power Curve

We'll sweep through wind speeds from 0 to 30 m/s in 0.5 m/s increments using a **list comprehension**.

In [ ]:
# Generate wind speed values: 0, 0.5, 1.0, ..., 30.0
wind_speeds = [v / 2 for v in range(0, 61)]  # 0 to 30 in 0.5 steps

# Calculate power at each speed
power_outputs = [wind_power(v) for v in wind_speeds]

print(f"Generated {len(wind_speeds)} data points")
print(f"Wind speed range: {wind_speeds[0]}–{wind_speeds[-1]} m/s")
print(f"Max power output: {max(power_outputs):,.1f} kW")

### Tabular Summary

Let's print a neat table showing power at integer wind speeds.

In [ ]:
print(f"{'Speed (m/s)':>12} {'Power (kW)':>12} {'Region':>10}")
print("-" * 38)

for v in range(0, 31):
    p = wind_power(v)

    # Determine operating region
    if v < cut_in_speed:
        region = "I"
    elif v <= rated_speed:
        region = "II"
    elif v <= cut_out_speed:
        region = "III"
    else:
        region = "IV"

    print(f"{v:>12.0f} {p:>12.1f} {region:>10}")

---
## 5. Visualizing the Power Curve

A table is useful, but the classic **power curve plot** is how wind engineers actually communicate turbine performance.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(wind_speeds, power_outputs, color='steelblue', linewidth=2.5)

# Mark key operating points
ax.axvline(cut_in_speed, color='gray', linestyle='--', alpha=0.6, label=f'Cut-in ({cut_in_speed} m/s)')
ax.axvline(rated_speed, color='orange', linestyle='--', alpha=0.6, label=f'Rated ({rated_speed} m/s)')
ax.axvline(cut_out_speed, color='indianred', linestyle='--', alpha=0.6, label=f'Cut-out ({cut_out_speed} m/s)')
ax.axhline(rated_power, color='orange', linestyle=':', alpha=0.4)

# Region labels
ax.text(2.0, 1800, 'Region I\n(No output)', ha='center', fontsize=9, color='gray')
ax.text(8.5, 1800, 'Region II\n(Cubic law)', ha='center', fontsize=9, color='steelblue')
ax.text(19.0, 1800, 'Region III\n(Rated)', ha='center', fontsize=9, color='orange')
ax.text(27.5, 1800, 'Region IV\n(Shutdown)', ha='center', fontsize=9, color='indianred')

ax.set_xlabel('Wind Speed (m/s)', fontsize=12)
ax.set_ylabel('Power Output (kW)', fontsize=12)
ax.set_title('Wind Turbine Power Curve — 2 MW Turbine (D = 90 m)', fontsize=13)
ax.set_xlim(0, 30)
ax.set_ylim(-50, 2300)
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

> **Key Insight: The Cubic Relationship**  
> Doubling the wind speed increases available power by a factor of **8** ($2^3 = 8$). This is why site selection is so critical — even small differences in average wind speed translate into large differences in energy production.

---
## 6. Simulating a Year of Wind Data

Real wind speeds at a site are not constant. They roughly follow a **Weibull distribution**, but we can approximate this with a simple model using Python's built-in `random` module.

In [ ]:
import random

random.seed(42)  # reproducible results

# Simulate 8760 hourly wind speeds (365 days × 24 hours)
hours_per_year = 365 * 24  # 8760
mean_wind_speed = 7.5      # m/s (typical good onshore site)

# Simple Weibull-like simulation using Gamma distribution
# (shape=2 approximates a Weibull with k≈2)
wind_data = [random.gammavariate(2.0, mean_wind_speed / 2.0)
             for _ in range(hours_per_year)]

# Basic statistics using only built-in Python
avg_speed = sum(wind_data) / len(wind_data)
min_speed = min(wind_data)
max_speed = max(wind_data)

# Median (sort and pick middle)
sorted_speeds = sorted(wind_data)
median_speed = sorted_speeds[len(sorted_speeds) // 2]

print(f"Simulated {hours_per_year:,} hours of wind data")
print(f"Mean wind speed:   {avg_speed:.2f} m/s")
print(f"Median wind speed: {median_speed:.2f} m/s")
print(f"Min wind speed:    {min_speed:.2f} m/s")
print(f"Max wind speed:    {max_speed:.2f} m/s")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

ax.hist(wind_data, bins=50, color='steelblue', alpha=0.7, edgecolor='white')
ax.axvline(avg_speed, color='indianred', linewidth=2, linestyle='--',
           label=f'Mean = {avg_speed:.1f} m/s')
ax.axvline(cut_in_speed, color='gray', linewidth=1, linestyle=':',
           label=f'Cut-in = {cut_in_speed} m/s')

ax.set_xlabel('Wind Speed (m/s)', fontsize=12)
ax.set_ylabel('Hours', fontsize=12)
ax.set_title('Simulated Annual Wind Speed Distribution', fontsize=13)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 7. Annual Energy Production (AEP)

The **Annual Energy Production** is arguably the single most important number in wind energy engineering. It determines revenue and project feasibility.

$$\text{AEP} = \sum_{t=1}^{8760} P(v_t) \times \Delta t$$

Since each time step is 1 hour, AEP in kWh is simply the sum of hourly power outputs.

In [ ]:
# Calculate power for each hour of the year
hourly_power = [wind_power(v) for v in wind_data]  # kW per hour

# Annual Energy Production
aep_kwh = sum(hourly_power)           # kWh (since each step is 1 hour)
aep_mwh = aep_kwh / 1000              # MWh
aep_gwh = aep_kwh / 1_000_000         # GWh

# Capacity factor = actual production / theoretical maximum
max_possible = rated_power * hours_per_year  # kWh if turbine ran at full power all year
capacity_factor = aep_kwh / max_possible

# Hours in each operating region
hours_region_I  = sum(1 for v in wind_data if v < cut_in_speed)
hours_region_II = sum(1 for v in wind_data if cut_in_speed <= v <= rated_speed)
hours_region_III = sum(1 for v in wind_data if rated_speed < v <= cut_out_speed)
hours_region_IV = sum(1 for v in wind_data if v > cut_out_speed)

print("=" * 50)
print("  ANNUAL ENERGY PRODUCTION REPORT")
print("=" * 50)
print(f"  AEP:             {aep_mwh:,.0f} MWh  ({aep_gwh:.2f} GWh)")
print(f"  Capacity Factor: {capacity_factor:.1%}")
print(f"  Avg hourly out:  {aep_kwh / hours_per_year:,.0f} kW")
print("-" * 50)
print(f"  Region I  (idle):    {hours_region_I:>5,} hours ({hours_region_I/hours_per_year:.1%})")
print(f"  Region II (partial): {hours_region_II:>5,} hours ({hours_region_II/hours_per_year:.1%})")
print(f"  Region III (rated):  {hours_region_III:>5,} hours ({hours_region_III/hours_per_year:.1%})")
print(f"  Region IV (cutout):  {hours_region_IV:>5,} hours ({hours_region_IV/hours_per_year:.1%})")
print("=" * 50)

> **Key Insight: Capacity Factor**  
> Typical onshore wind farms have capacity factors of 25–35%. Offshore sites can reach 40–50% due to stronger, more consistent winds. A capacity factor around 30% is considered good for an onshore site.

---
## 8. Monthly Energy Breakdown

Wind is not uniform across the year. Let's compute monthly production using loops and slicing.

In [ ]:
# Days in each month (non-leap year)
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

monthly_energy = []   # MWh per month
monthly_avg_wind = [] # Average wind speed per month

hour_start = 0
for month_idx in range(12):
    hours_in_month = days_per_month[month_idx] * 24
    hour_end = hour_start + hours_in_month

    # Slice the data for this month
    month_wind = wind_data[hour_start:hour_end]
    month_power = hourly_power[hour_start:hour_end]

    energy_mwh = sum(month_power) / 1000
    avg_wind = sum(month_wind) / len(month_wind)

    monthly_energy.append(energy_mwh)
    monthly_avg_wind.append(avg_wind)

    hour_start = hour_end

# Print monthly summary
print(f"{'Month':>5} {'Avg Wind (m/s)':>15} {'Energy (MWh)':>13}")
print("-" * 36)
for i in range(12):
    print(f"{month_names[i]:>5} {monthly_avg_wind[i]:>15.2f} {monthly_energy[i]:>13.1f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Monthly energy bar chart
ax1.bar(month_names, monthly_energy, color='steelblue', alpha=0.8, edgecolor='white')
ax1.set_ylabel('Energy (MWh)', fontsize=11)
ax1.set_title('Monthly Energy Production', fontsize=12)
ax1.grid(True, alpha=0.3, axis='y')

# Monthly average wind speed
ax2.bar(month_names, monthly_avg_wind, color='indianred', alpha=0.8, edgecolor='white')
ax2.axhline(avg_speed, color='gray', linestyle='--', alpha=0.6,
            label=f'Annual mean = {avg_speed:.1f} m/s')
ax2.set_ylabel('Avg Wind Speed (m/s)', fontsize=11)
ax2.set_title('Monthly Average Wind Speed', fontsize=12)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## 9. Sensitivity Analysis: How Site Wind Speed Affects Revenue

One of the most powerful uses of simulation is answering **"what if?"** questions. Let's vary the mean wind speed and see how AEP changes.

In [ ]:
random.seed(123)  # fresh seed for reproducibility

mean_speeds = [5.0, 6.0, 7.0, 7.5, 8.0, 9.0, 10.0]
results = []  # list of dictionaries

for mean_v in mean_speeds:
    # Simulate one year of wind at this mean speed
    year_wind = [random.gammavariate(2.0, mean_v / 2.0)
                 for _ in range(hours_per_year)]
    year_power = [wind_power(v) for v in year_wind]
    aep = sum(year_power) / 1000  # MWh
    cf = sum(year_power) / (rated_power * hours_per_year)

    results.append({
        'mean_speed': mean_v,
        'aep_mwh': aep,
        'capacity_factor': cf
    })

# Print results
print(f"{'Mean Speed':>11} {'AEP (MWh)':>10} {'Cap. Factor':>12}")
print("-" * 36)
for r in results:
    print(f"{r['mean_speed']:>11.1f} {r['aep_mwh']:>10,.0f} {r['capacity_factor']:>12.1%}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

speeds = [r['mean_speed'] for r in results]
aeps = [r['aep_mwh'] for r in results]
cfs = [r['capacity_factor'] * 100 for r in results]

color1 = 'steelblue'
ax.bar(speeds, aeps, width=0.6, color=color1, alpha=0.8, edgecolor='white')
ax.set_xlabel('Mean Wind Speed (m/s)', fontsize=12)
ax.set_ylabel('AEP (MWh)', fontsize=12, color=color1)
ax.tick_params(axis='y', labelcolor=color1)

# Second y-axis for capacity factor
ax2 = ax.twinx()
ax2.plot(speeds, cfs, color='indianred', marker='o', linewidth=2, markersize=7)
ax2.set_ylabel('Capacity Factor (%)', fontsize=12, color='indianred')
ax2.tick_params(axis='y', labelcolor='indianred')

ax.set_title('Annual Energy Production vs. Mean Wind Speed', fontsize=13)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

> **Example: Civil Engineering**  
> At a typical electricity price of €50/MWh, the difference between a 6 m/s site and an 8 m/s site can mean **hundreds of thousands of euros** per year per turbine. This is why wind resource assessment — measuring wind speeds for 1–2 years before construction — is a critical step in wind farm development.

---
## 10. [PRACTICE] Your Turn: Explore a Different Turbine

> **Key Insight: Challenge**  
> Modify the simulation to analyze a **smaller turbine** with the following specs:
> - Rotor diameter: 52 m
> - Rated power: 850 kW
> - Cut-in speed: 3.5 m/s
> - Rated speed: 15 m/s
> - Cut-out speed: 25 m/s
>
> **Tasks:**
> 1. Calculate the swept area
> 2. Generate the power curve (use a list comprehension)
> 3. Estimate the AEP using `wind_data` from Section 6
> 4. Compare the capacity factor with the 2 MW turbine
>
> **Hint:** You can reuse the `wind_power` function — just pass different parameters!

In [ ]:
# YOUR CODE HERE
# Step 1: Define the new turbine parameters


# Step 2: Calculate swept area


# Step 3: Generate power curve and compute AEP


# Step 4: Print results and compare


---
## Summary

In this notebook we used only **Week 1 Python fundamentals** to build a complete wind energy simulation:

| Python Concept | How We Used It |
|:---|:---|
| Variables & arithmetic | Turbine parameters, physics equations |
| Functions with defaults | `wind_power()` — reusable for any turbine |
| `if`/`elif`/`else` | Four operating regions of the power curve |
| Lists & list comprehensions | Wind speed arrays, hourly power calculations |
| `for` loops & `range` | Monthly breakdown, sensitivity analysis |
| Dictionaries | Storing structured results |
| String formatting (f-strings) | Clean tabular output |
| `import math`, `import random` | Built-in modules for π and random data |

> **Key Insight: Computational Thinking**  
> We decomposed a real engineering problem ("how much energy will this turbine produce?") into small, testable Python functions and built up the full analysis step by step. This is the essence of computational thinking.

---

### Questions?

**Dr. Eyuphan Koc**  
eyuphan.koc@bogazici.edu.tr